# Wrangling Countries & UN Regions
## By: Scott Kustes

### Objective:
Wrangle UN regions and subregions and associated countries for insertion into website database.

#### Dataset:
The original dataset was downloaded here: https://unstats.un.org/unsd/methodology/m49/

#### Contents
- <a href='#gather'>Data Gathering</a>
- <a href='#assess1'>Assess, Part 1</a>
- <a href='#clean1'>Clean, Part 1</a>
- <a href='#assess2'>Assess, Part 2</a>
- <a href='#clean2'>Clean, Part 2</a>
- <a href='#extract-un'>UN Regional Hierarchy Extraction</a>
- <a href='#assess3'>Assess, Part 3</a>
- <a href='#clean3'>Clean, Part 3</a>
- <a href='#extract-groups'>UN Groupings Extraction</a>
- <a href='#assess4'>Assess, Part 4</a>
- <a href='#clean4'>Clean, Part 4</a>
- <a href='#final'>Finished Dataframes</a>
- <a href='#db'>Insert into Database</a>

In [1]:
# Import necessary packages
import pandas as pd

<a id='gather'></a>
## Gather

In [2]:
countries = pd.read_csv( 'countries.csv' )
countries.sample(5)

,Global Code,Global Name,Region Code,Region Name,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,Common Name,Official Name,Capital,M49 Code,ISO-alpha3 Code,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),Small Island Developing States (SIDS),Developed / Developing Countries
56,1,World,19.0,Americas,419.0,Latin America and the Caribbean,29.0,Caribbean,Cuba,Republic of Cuba,Havana,192,CUB,NaN,NaN,x,Developing
198,1,World,142.0,Asia,35.0,South-eastern Asia,NaN,NaN,Singapore,Republic of Singapore,Singapore,702,SGP,NaN,NaN,x,Developing
132,1,World,2.0,Africa,202.0,Sub-Saharan Africa,14.0,Eastern Africa,Malawi,Republic of Malawi,Lilongwe,454,MWI,x,x,NaN,Developing
53,1,World,19.0,Americas,419.0,Latin America and the Caribbean,13.0,Central America,Costa Rica,Republic of Costa Rica,San José,188,CRI,NaN,NaN,NaN,Developing
57,1,World,19.0,Americas,419.0,Latin America and the Caribbean,29.0,Caribbean,Curaçao,Country of Curaçao,Willemstad,531,CUW,NaN,NaN,x,Developing


<a id='assess1'></a>
## Assess, Part 1

In [3]:
countries['Global Code'].unique()

array([1], dtype=int64)

In [4]:
countries['Global Name'].unique()

array(['World'], dtype=object)

### Issues Found:
1) Drop columns: `Global Code` and `Global Name` - only 1 unique value

2) Rename columns: replace spaces with underscores, replace uppercase with lowercase

<a id='clean1'></a>
## Clean, Part 1
### 1) Drop columns `Global Code` and `Global Name`

Drop `Global Code` and `Global Name` columns due to each having only 1 unique value (1 and World, respectively).

#### Code

In [5]:
countries.drop( columns=['Global Code','Global Name'], axis=1, inplace=True )

#### Test

In [6]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 15 columns):
Region Code                                248 non-null float64
Region Name                                248 non-null object
Sub-region Code                            248 non-null float64
Sub-region Name                            248 non-null object
Intermediate Region Code                   108 non-null float64
Intermediate Region Name                   108 non-null object
Common Name                                249 non-null object
Official Name                              249 non-null object
Capital                                    243 non-null object
M49 Code                                   249 non-null int64
ISO-alpha3 Code                            248 non-null object
Least Developed Countries (LDC)            47 non-null object
Land Locked Developing Countries (LLDC)    32 non-null object
Small Island Developing States (SIDS)      53 non-null object
Developed / De

### 2) Rename Columns
Replace spaces with underscores, replace uppercase letters with lowercase

#### Code

In [7]:
# Dictionary of new column names
column_names = {
    'Region Code': 'region_code',
    'Region Name': 'region_name',
    'Sub-region Code': 'subregion_code',
    'Sub-region Name': 'subregion_name',
    'Intermediate Region Code': 'intermediate_region_code',
    'Intermediate Region Name': 'intermediate_region_name',
    'Common Name': 'common_name',
    'Official Name': 'official_name',
    'Capital': 'capital',
    'Territory of': 'territory_of',
    'M49 Code': 'un_m49',
    'ISO-alpha3 Code': 'iso_alpha3',
    'Least Developed Countries (LDC)': 'least_developed_countries',
    'Land Locked Developing Countries (LLDC)': 'landlocked_developing_countries',
    'Small Island Developing States (SIDS)': 'small_island_developing_states',
    'Developed / Developing Countries': 'developed_developing_countries'
}

countries.rename( mapper=column_names, axis=1, inplace=True )

#### Test

In [8]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 15 columns):
region_code                        248 non-null float64
region_name                        248 non-null object
subregion_code                     248 non-null float64
subregion_name                     248 non-null object
intermediate_region_code           108 non-null float64
intermediate_region_name           108 non-null object
common_name                        249 non-null object
official_name                      249 non-null object
capital                            243 non-null object
un_m49                             249 non-null int64
iso_alpha3                         248 non-null object
least_developed_countries          47 non-null object
landlocked_developing_countries    32 non-null object
small_island_developing_states     53 non-null object
developed_developing_countries     248 non-null object
dtypes: float64(3), int64(1), object(11)
memory usage: 29.3+ KB


<a id='assess2'></a>
## Assess, Part 2

In [9]:
countries['region_code'].unique()

array([  2., 142., 150.,   9.,  19.,  nan])

In [10]:
countries['region_name'].unique()

array(['Africa', 'Asia', 'Europe', 'Oceania', 'Americas', nan],
      dtype=object)

In [11]:
countries[ countries['region_code'].isnull() ]

,region_code,region_name,subregion_code,subregion_name,intermediate_region_code,intermediate_region_name,common_name,official_name,capital,un_m49,iso_alpha3,least_developed_countries,landlocked_developing_countries,small_island_developing_states,developed_developing_countries
9,NaN,NaN,NaN,NaN,NaN,NaN,Antarctica,Antarctica,NaN,10,ATA,NaN,NaN,NaN,NaN


In [12]:
countries['developed_developing_countries'].unique()

array(['Developing', 'Developed', nan], dtype=object)

In [13]:
countries['intermediate_region_name'].unique()

array(['Western Africa', nan, 'Middle Africa', 'Caribbean',
       'South America', 'Central America', 'Southern Africa',
       'Eastern Africa', 'Channel Islands'], dtype=object)

In [14]:
countries['region_name'].unique()

array(['Africa', 'Asia', 'Europe', 'Oceania', 'Americas', nan],
      dtype=object)

In [15]:
countries['subregion_name'].unique()

array(['Sub-Saharan Africa', 'Southern Asia', 'Northern Europe',
       'Southern Europe', 'Northern Africa', 'Polynesia',
       'Latin America and the Caribbean', nan, 'Western Asia',
       'Australia and New Zealand', 'Western Europe', 'Eastern Europe',
       'Northern America', 'South-eastern Asia', 'Eastern Asia',
       'Melanesia', 'Micronesia', 'Central Asia'], dtype=object)

### Issues Found:
1) Replace NaN with 0 in `region_code`, `subregion_code`, and `intermediate_region_code`

2) Replace NaN with 0 and 'x' with 1 in `least_developed_countries`, `landlocked_developing_countries`, and `small_island_developing_states`

3) Set datatypes:
- <strong>str:</strong> `region_code`, `subregion_code`, `intermediate_region_code`, padded with zeroes to three characters</li>
- <strong>bool:</strong> `least_developed_countries`, `landlocked_developing_countries`, `small_island_developing_states`

4) Set Antarctica's `region_code`, `subregion_code`, and `intermediate_region_code` to 999.

5) Replace NaN with empty string in `capital`

<a id='clean2'></a>
## Clean, Part 2
### 1) Fix Values in Columns with Region Codes
Replace NaN with 0 in region_code, subregion_code, and intermediate_region_code

#### Code

In [16]:
countries['region_code'].fillna( 0, inplace=True )
countries['subregion_code'].fillna( 0, inplace=True )
countries['intermediate_region_code'].fillna( 0, inplace=True )

#### Test

In [17]:
countries['region_code'].unique()

array([  2., 142., 150.,   9.,  19.,   0.])

In [18]:
countries['subregion_code'].unique()

array([202.,  34., 154.,  39.,  15.,  61., 419.,   0., 145.,  53., 155.,
       151.,  21.,  35.,  30.,  54.,  57., 143.])

In [19]:
countries['intermediate_region_code'].unique()

array([ 11.,   0.,  17.,  29.,   5.,  13.,  18.,  14., 830.])

### 2) Fix Values in 'Other Groupings'
Replace NaN with 0 and 'x' with 1 in `least_developed_countries`, `landlocked_developing_countries`, and `small_island_developing_states`

#### Code

In [20]:
countries['least_developed_countries'].fillna( 0, inplace=True )
countries['least_developed_countries'].replace( 'x', 1, inplace=True )

countries['landlocked_developing_countries'].fillna( 0, inplace=True )
countries['landlocked_developing_countries'].replace( 'x', 1, inplace=True )

countries['small_island_developing_states'].fillna( 0, inplace=True )
countries['small_island_developing_states'].replace( 'x', 1, inplace=True )

#### Test

In [21]:
countries['least_developed_countries'].unique()

array([0, 1], dtype=int64)

In [22]:
countries['landlocked_developing_countries'].unique()

array([0, 1], dtype=int64)

In [23]:
countries['small_island_developing_states'].unique()

array([0, 1], dtype=int64)

### 3) Set Datatypes
- <strong>str:</strong> `region_code`, `subregion_code`, `intermediate_region_code`, `un_m49` padded with zeroes to 3 characters
- <strong>bool:</strong> `least_developed_countries`, `landlocked_developing_countries`, and `small_island_developing_states`

#### Code

In [24]:
# Set str columns, convert to int64 first to remove decimal and following zero
countries['region_code'] = countries['region_code'].astype( 'int64' ).astype( 'str' )
countries['subregion_code'] = countries['subregion_code'].astype( 'int64' ).astype( 'str' )
countries['intermediate_region_code'] = countries['intermediate_region_code'].astype( 'int64' ).astype( 'str' )
countries['un_m49'] = countries['un_m49'].astype( 'int64' ).astype( 'str' )

# Pad with zeroes to three characters
countries['region_code'] = countries['region_code'].apply( lambda x: x.zfill(3) )
countries['subregion_code'] = countries['subregion_code'].apply( lambda x: x.zfill(3) )
countries['intermediate_region_code'] = countries['intermediate_region_code'].apply( lambda x: x.zfill(3) )
countries['un_m49'] = countries['un_m49'].apply( lambda x: x.zfill(3) )

In [25]:
# Set bool columns
countries['least_developed_countries'] = countries['least_developed_countries'].astype( 'bool' )
countries['landlocked_developing_countries'] = countries['landlocked_developing_countries'].astype( 'bool' )
countries['small_island_developing_states'] = countries['small_island_developing_states'].astype( 'bool' )

#### Test

In [26]:
countries['region_code'].unique()

array(['002', '142', '150', '009', '019', '000'], dtype=object)

In [27]:
countries['subregion_code'].unique()

array(['202', '034', '154', '039', '015', '061', '419', '000', '145',
       '053', '155', '151', '021', '035', '030', '054', '057', '143'],
      dtype=object)

In [28]:
countries['intermediate_region_code'].unique()

array(['011', '000', '017', '029', '005', '013', '018', '014', '830'],
      dtype=object)

In [29]:
countries['un_m49'].sample(5)

234    826
52     184
181    643
27     068
163    578
Name: un_m49, dtype: object

In [30]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 15 columns):
region_code                        249 non-null object
region_name                        248 non-null object
subregion_code                     249 non-null object
subregion_name                     248 non-null object
intermediate_region_code           249 non-null object
intermediate_region_name           108 non-null object
common_name                        249 non-null object
official_name                      249 non-null object
capital                            243 non-null object
un_m49                             249 non-null object
iso_alpha3                         248 non-null object
least_developed_countries          249 non-null bool
landlocked_developing_countries    249 non-null bool
small_island_developing_states     249 non-null bool
developed_developing_countries     248 non-null object
dtypes: bool(3), object(12)
memory usage: 24.2+ KB


### 4) Set Antarctica's Region, Subregion, and Intermediate Region Codes to 999

#### Code

In [31]:
countries.loc[9,'region_code'] = '999'
countries.loc[9,'subregion_code'] = '999'
countries.loc[9,'intermediate_region_code'] = '999'

#### Test

In [32]:
countries.query( 'common_name == "Antarctica"' )

,region_code,region_name,subregion_code,subregion_name,intermediate_region_code,intermediate_region_name,common_name,official_name,capital,un_m49,iso_alpha3,least_developed_countries,landlocked_developing_countries,small_island_developing_states,developed_developing_countries
9,999,NaN,999,NaN,999,NaN,Antarctica,Antarctica,NaN,010,ATA,False,False,False,NaN


### 5) Fix NaN in Capital
Set NaN values to empty string.

#### Code

In [33]:
countries['capital'].fillna( '', inplace=True )

#### Test

In [34]:
countries[ countries['capital'].isnull() ]

,region_code,region_name,subregion_code,subregion_name,intermediate_region_code,intermediate_region_name,common_name,official_name,capital,un_m49,iso_alpha3,least_developed_countries,landlocked_developing_countries,small_island_developing_states,developed_developing_countries


<a id='extract-un'></a>
## UN Regional Hierarchy Extraction
Extract UN Regional hierarchy and rationalize for insertion into website database.

### Create UN Region Hierarchy
In the UN Geoscheme, a region is made up of 1 or more subregions and a subregion is made up of 0 or more intermediate regions. Countries/areas are assigned at either the subregional or intermediate regional level (with Antarctica as the sole exception, having no regional assignment). Hierarchy levels are as follows: `Region -> Subregion -> Intermediate Region`

Antarctica is assigned to `region_code` 999.

The `countries` dataframe contains redundant information across the region, subregion, and intermediate region columns. Extract these values to create a hierarchy of UN regions in a new dataframe called `regions`. This new dataframe will contain 3 columns: 
- `region_code`: the value from the `region_code`, `subregion_code`, or `intermediate_region_code` column
- `region_name`: the value from the `region_name`, `subregion_name`, or `intermediate_region_name` column
- `parent_region_code`: the `region_code` for the region or subregion one level above in the hierarchy

In the `countries` dataframe, create a new column called `un_region` to hold the subregion or intermediate region to which each country is assigned.

#### Code

In [35]:
# Create an empty dataframe for holding the regions
regions = pd.DataFrame( columns=['region_code','region_name','parent_region_code'] )

# Iterate through the countries to extract the UN region information
for row in countries.itertuples():
    # Get the region information from this row
    region_code = row.region_code
    region_name = row.region_name if region_code != '999' else 'Antarctica' # Antarctica is the sole exception so make it a top-level region
    subregion_code = row.subregion_code
    subregion_name = row.subregion_name
    intermediate_region_code = row.intermediate_region_code
    intermediate_region_name = row.intermediate_region_name
    
    # If region doesn't exist in regions dataframe already, add it
    if region_code not in regions['region_code'].unique():
        regions = regions.append( { 'region_code': region_code, 
                                    'region_name': region_name, 
                                    'parent_region_code': '000' }, 
                                    ignore_index=True )
    
    # If subregion doesn't exist in regions dataframe already, add it
    if ( subregion_code != '000' ) & ( subregion_code not in regions['region_code'].unique() ):
        regions = regions.append( { 'region_code': subregion_code, 
                                    'region_name': subregion_name, 
                                    'parent_region_code': region_code }, 
                                    ignore_index=True )

    # If intermediate region doesn't exist in regions dataframe already, add it
    if ( intermediate_region_code != '000' ) & ( intermediate_region_code not in regions['region_code'].unique() ):
        regions = regions.append( { 'region_code': intermediate_region_code, 
                                    'region_name': intermediate_region_name, 
                                    'parent_region_code': subregion_code }, 
                                    ignore_index=True )

In [36]:
# Function to populate un_region field in countries dataframe
# All assignments to regions are at the subregion or intermediate region level
# If intermediate_region_code is not 000, return the intermediate_region_code
# Otherwise return subregion_code
def get_un_region( row ):
    # If intermediate_region_code is not 000, return it
    return row['intermediate_region_code'] if row['intermediate_region_code'] != '000' else row['subregion_code']

countries['un_region'] = countries.apply( get_un_region, axis=1 )

#### Test

In [37]:
regions.sort_values( 'parent_region_code' )

,region_code,region_name,parent_region_code
0,002,Africa,000
12,019,Americas,000
9,009,Oceania,000
5,150,Europe,000
15,999,Antarctica,000
3,142,Asia,000
8,015,Northern Africa,002
1,202,Sub-Saharan Africa,002
28,057,Micronesia,009
27,054,Melanesia,009


In [38]:
countries[['subregion_code','intermediate_region_code','un_region']].sample(10)

,subregion_code,intermediate_region_code,un_region
187,021,000,021
61,202,014,014
124,202,011,011
156,419,013,013
147,419,029,029
112,030,000,030
228,143,000,143
97,419,029,029
215,145,000,145
190,039,000,039


<a id='assess3'></a>
## Assess, Part 3

In [39]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 16 columns):
region_code                        249 non-null object
region_name                        248 non-null object
subregion_code                     249 non-null object
subregion_name                     248 non-null object
intermediate_region_code           249 non-null object
intermediate_region_name           108 non-null object
common_name                        249 non-null object
official_name                      249 non-null object
capital                            249 non-null object
un_m49                             249 non-null object
iso_alpha3                         248 non-null object
least_developed_countries          249 non-null bool
landlocked_developing_countries    249 non-null bool
small_island_developing_states     249 non-null bool
developed_developing_countries     248 non-null object
un_region                          249 non-null object
dtypes: bool(3), obje

In [40]:
countries[ countries['iso_alpha3'].isnull() ]

,region_code,region_name,subregion_code,subregion_name,intermediate_region_code,intermediate_region_name,common_name,official_name,capital,un_m49,iso_alpha3,least_developed_countries,landlocked_developing_countries,small_island_developing_states,developed_developing_countries,un_region
192,150,Europe,154,Northern Europe,830,Channel Islands,Sark,Sark,,680,NaN,False,False,False,Developed,830


In [41]:
countries[ countries['developed_developing_countries'].isnull() ]

,region_code,region_name,subregion_code,subregion_name,intermediate_region_code,intermediate_region_name,common_name,official_name,capital,un_m49,iso_alpha3,least_developed_countries,landlocked_developing_countries,small_island_developing_states,developed_developing_countries,un_region
9,999,NaN,999,NaN,999,NaN,Antarctica,Antarctica,,010,ATA,False,False,False,NaN,999


### Issues Found:
#### Tidiness
1) Drop 6 UN region columns: `region_code`,`region_name`,`subregion_code`,`subregion_name`,`intermediate_region_code`,`intermediate_region_name`

2) Create two columns, `developed` and `developing`, from `developed_developing_countries` column. 

3) Set datatype of `developed` and `developing` to bool.

4) Drop `developed_developing_countries`.

#### Quality
5) Replace NaN in `iso_alpha3` with 'NA'. The island of Sark has no ISO Alpha3 code.

<a id='clean3'></a>
## Clean, Part 3

### 1) Drop 6 UN region columns
Drop `region_code`,`region_name`,`subregion_code`,`subregion_name`,`intermediate_region_code`, and `intermediate_region_name`. These columns are no longer needed. This information is now contained in the `regions` dataframe to which each country is associated through the `un_region` column.

#### Code

In [42]:
countries.drop( columns=['region_code','region_name','subregion_code','subregion_name','intermediate_region_code','intermediate_region_name'], axis=1, inplace=True )

#### Test

In [43]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 10 columns):
common_name                        249 non-null object
official_name                      249 non-null object
capital                            249 non-null object
un_m49                             249 non-null object
iso_alpha3                         248 non-null object
least_developed_countries          249 non-null bool
landlocked_developing_countries    249 non-null bool
small_island_developing_states     249 non-null bool
developed_developing_countries     248 non-null object
un_region                          249 non-null object
dtypes: bool(3), object(7)
memory usage: 14.5+ KB


### 2) Create `developed` and `developing` Columns

`developed` will contain 1 if `developed_developing_countries` contains 'Developed', 0 otherwise<br/>
`developing` will contain 1 if `developed_developing_countries` contains 'Developing', 0 otherwise

#### Code

In [44]:
countries['developed'] = countries['developed_developing_countries'].apply( lambda x: 1 if x == 'Developed' else 0 )
countries['developing'] = countries['developed_developing_countries'].apply( lambda x: 1 if x == 'Developing' else 0 )

#### Test

In [45]:
countries[['developed','developing','developed_developing_countries']].sample(5)

,developed,developing,developed_developing_countries
170,0,1,Developing
154,0,1,Developing
195,1,0,Developed
81,0,1,Developing
210,0,1,Developing


### 3) Set datatype for `developed` and `developing` Columns to bool

#### Code

In [46]:
countries['developed'] = countries['developed'].astype('bool')
countries['developing'] = countries['developing'].astype('bool')

#### Test

In [47]:
countries['developed'].value_counts()

False    183
True      66
Name: developed, dtype: int64

In [48]:
countries['developing'].value_counts()

True     182
False     67
Name: developing, dtype: int64

In [49]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 12 columns):
common_name                        249 non-null object
official_name                      249 non-null object
capital                            249 non-null object
un_m49                             249 non-null object
iso_alpha3                         248 non-null object
least_developed_countries          249 non-null bool
landlocked_developing_countries    249 non-null bool
small_island_developing_states     249 non-null bool
developed_developing_countries     248 non-null object
un_region                          249 non-null object
developed                          249 non-null bool
developing                         249 non-null bool
dtypes: bool(5), object(7)
memory usage: 15.0+ KB


### 4) Drop `developed_developing_countries` Column

#### Code

In [50]:
countries.drop( columns=['developed_developing_countries'], inplace=True )

#### Test

In [51]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 11 columns):
common_name                        249 non-null object
official_name                      249 non-null object
capital                            249 non-null object
un_m49                             249 non-null object
iso_alpha3                         248 non-null object
least_developed_countries          249 non-null bool
landlocked_developing_countries    249 non-null bool
small_island_developing_states     249 non-null bool
un_region                          249 non-null object
developed                          249 non-null bool
developing                         249 non-null bool
dtypes: bool(5), object(6)
memory usage: 13.0+ KB


### 5) Replace NaN in `iso_alpha3` with empty string

The island of Sark has no ISO Alpha3 code.

#### Code

In [52]:
countries['iso_alpha3'].fillna( '', inplace=True )

#### Test

In [53]:
countries[ countries['iso_alpha3'].isnull() ]

,common_name,official_name,capital,un_m49,iso_alpha3,least_developed_countries,landlocked_developing_countries,small_island_developing_states,un_region,developed,developing


<a id='extract-groups'></a>
## UN Groupings Extraction
Extract and rationalize information in the five "Other Groupings" columns: `least_developed_countries`, `landlocked_developing_countries`, `small_island_developing_states`, `developed`, and `developing`.  This information will be stored in 2 database tables. 

The first table will contain five grouping titles: 'Least Developed Countries', 'Landlocked Developing Countries', 'Small Island Developing States', 'Developed', and 'Developing'.

The second table will contain mappings of country ID to grouping ID.

#### Code

In [54]:
# Create a dataframe for the Other Groupings
group_names = ['Least Developed Countries', 'Landlocked Developing Countries', 'Small Island Developing States', 'Developed Countries', 'Developing Countries']
un_groupings = pd.DataFrame( data=group_names, columns=['society_group_name'] )

# Set the Society Group Type to 1, which is the ID for "UN Stats Grouping" in society_group_types table
# At the time of this analysis, this is the only entry in society_group_types table
un_groupings['society_group_type'] = 1

In [55]:
# Get the indexes for the countries in each grouping
least_developed = countries.query( 'least_developed_countries == True' ).index.to_list()
landlocked = countries.query( 'landlocked_developing_countries == True' ).index.to_list()
small_island = countries.query( 'small_island_developing_states == True' ).index.to_list()
developed = countries.query( 'developed == True' ).index.to_list()
developing = countries.query( 'developing == True' ).index.to_list()

In [56]:
# Create a blank dataframe to contain the country to grouping mappings
country_to_group = pd.DataFrame( columns=['country_id', 'group_id'] )

groups = ['least_developed','landlocked','small_island','developed','developing']
group_id = 0
for variable_name in groups:
    for value in eval( variable_name ):
        # Add 1 to the values being appended because at the time of this analysis
        # the database tables are empty and the id values will end up being 1 more than the index value
        # due to SQL being 1-indexed instead of 0-indexed
        country_to_group = country_to_group.append( { 'country_id': value+1,
                                                      'group_id': group_id+1 },
                                                      ignore_index=True )
    group_id += 1

#### Test

In [57]:
un_groupings

,society_group_name,society_group_type
0,Least Developed Countries,1
1,Landlocked Developing Countries,1
2,Small Island Developing States,1
3,Developed Countries,1
4,Developing Countries,1


In [58]:
country_to_group.sample(10)

,country_id,group_id
51,27,1
168,126,3
61,120,1
351,215,4
26,139,0
292,135,4
139,24,3
249,69,4
170,128,3
237,53,4


In [59]:
# Found this one by luck
country_to_group.query( 'country_id == 230' )

,country_id,group_id
42,230,0
129,230,2
365,230,4


In [60]:
# Is True in the correct columns?
countries.query( 'index == 230' )

,common_name,official_name,capital,un_m49,iso_alpha3,least_developed_countries,landlocked_developing_countries,small_island_developing_states,un_region,developed,developing
230,Tuvalu,Ellice Islands,Funafuti,798,TUV,True,False,True,061,False,True


<a id='assess4'></a>
## Assess, Part 4

In [61]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 11 columns):
common_name                        249 non-null object
official_name                      249 non-null object
capital                            249 non-null object
un_m49                             249 non-null object
iso_alpha3                         249 non-null object
least_developed_countries          249 non-null bool
landlocked_developing_countries    249 non-null bool
small_island_developing_states     249 non-null bool
un_region                          249 non-null object
developed                          249 non-null bool
developing                         249 non-null bool
dtypes: bool(5), object(6)
memory usage: 13.0+ KB


### Issues Found:
#### Quality
1) Remove quotes from Common Name, Official Name and Capital. Several entries contain double quotes (ex: "Bonaire, Sint Eustatius, and Saba") due to commas in string breaking CSV formatting on input. Remove quotes.

2) Add `society_type_id` feature and set to 1 (society_type_id for "Country"). 

#### Tidiness
3) Drop 5 UN grouping columns: `least_developed_countries`,`landlocked_developing_countries`,`small_island_developing_states`,`developed`,`developing`

<a id='clean4'></a>
## Clean, Part 4

### 1) Remove quotes from Official Names

Several entries contain " in official name (ex: "Bonaire, Sint Eustatius, and Saba") due to commas in field breaking CSV formatting on input. Remove quotes.

Also, the official name for Sri Lanka contains an odd space character that breaks on read. Fix it.

#### Code

In [62]:
countries[ countries['official_name'].str.contains('"') | countries['capital'].str.contains('"') ]

,common_name,official_name,capital,un_m49,iso_alpha3,least_developed_countries,landlocked_developing_countries,small_island_developing_states,un_region,developed,developing
0,"""Saint Helena, Ascension, and Tristan da Cunha""","""Saint Helena, Ascension, and Tristan da Cunha""",Jamestown,654,SHN,False,False,False,011,False,True
25,BES Islands,"""Bonaire, Sint Eustatius, and Saba""",Kralendijk,535,BES,False,False,True,029,False,True
238,USA,United States of America,"""Washington, D.C.""",840,USA,False,False,False,021,True,False


In [63]:
countries.loc[0,'official_name'] = "Saint Helena, Ascension, and Tristan da Cunha"
countries.loc[0,'common_name'] = "Saint Helena, Ascension, and Tristan da Cunha"
countries.loc[25,'official_name'] = "Bonaire, Sint Eustatius, and Saba"
countries.loc[209,'official_name'] = "Democratic Socialist Republic of Sri Lanka"
countries.loc[238,'capital'] = "Washington, D.C."

#### Test

In [64]:
countries.query( '( common_name == "BES Islands" ) | ( common_name == "Saint Helena, Ascension, and Tristan da Cunha") | ( common_name == "Sri Lanka" ) | ( common_name == "USA" )' )

,common_name,official_name,capital,un_m49,iso_alpha3,least_developed_countries,landlocked_developing_countries,small_island_developing_states,un_region,developed,developing
0,"Saint Helena, Ascension, and Tristan da Cunha","Saint Helena, Ascension, and Tristan da Cunha",Jamestown,654,SHN,False,False,False,011,False,True
25,BES Islands,"Bonaire, Sint Eustatius, and Saba",Kralendijk,535,BES,False,False,True,029,False,True
209,Sri Lanka,Democratic Socialist Republic of Sri Lanka,Sri Jayawardenepura Kotte,144,LKA,False,False,False,034,False,True
238,USA,United States of America,"Washington, D.C.",840,USA,False,False,False,021,True,False


### 2) Add `society_type_id` and Set to 1
Create a new feature `society_type_id` and set it to 1 for all countries. This corresponds to the society_type_id for "Country".

#### Code

In [65]:
countries['society_type_id'] = 1

#### Test

In [66]:
countries.sample(5)

,common_name,official_name,capital,un_m49,iso_alpha3,least_developed_countries,landlocked_developing_countries,small_island_developing_states,un_region,developed,developing,society_type_id
138,Martinique,Martinique,Fort-de-France,474,MTQ,False,False,False,029,False,True,1
6,Andorra,Principality of Andorra,Andorra la Vella,020,AND,False,False,False,039,True,False,1
233,United Arab Emirates,United Arab Emirates,Abu Dhabi,784,ARE,False,False,False,145,False,True,1
200,Slovakia,Slovak Republic,Bratislava,703,SVK,False,False,False,151,True,False,1
132,Malawi,Republic of Malawi,Lilongwe,454,MWI,True,True,False,014,False,True,1


### 3) Drop 5 UN grouping columns
Drop `least_developed_countries`, `landlocked_developing_countries`, `small_island_developing_states`, `developed`, and `developing`. These columns are no longer needed. This information is now contained in the `un_groupings` and `country_to_group` dataframes.

#### Code

In [67]:
countries.drop( columns=['least_developed_countries','landlocked_developing_countries','small_island_developing_states','developed','developing'], axis=1, inplace=True )

#### Test

In [68]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 7 columns):
common_name        249 non-null object
official_name      249 non-null object
capital            249 non-null object
un_m49             249 non-null object
iso_alpha3         249 non-null object
un_region          249 non-null object
society_type_id    249 non-null int64
dtypes: int64(1), object(6)
memory usage: 13.7+ KB


<a id='final'></a>
## Finished Dataframes

These dataframes are ready for database insertion.

In [69]:
# Sort by common_name, then output to csv
countries.sort_values( 'common_name', inplace=True )
countries.to_csv( 'countries_clean.csv', encoding='utf-16', index=False )
countries.sample(5)

,common_name,official_name,capital,un_m49,iso_alpha3,un_region,society_type_id
221,Timor-Leste,Democratic Republic of Timor-Leste,Dili,626,TLS,035,1
162,Northern Mariana Islands,Commonwealth of the Northern Mariana Islands,Saipan,580,MNP,057,1
205,South Georgia and the South Sandwich Islands,South Georgia and the South Sandwich Islands,King Edward Point,239,SGS,005,1
185,Saint Lucia,Saint Lucia,Castries,662,LCA,029,1
242,Venezuela,Bolivarian Republic of Venezuela,Caracas,862,VEN,005,1


In [70]:
# Sort by parent_region_code, then output to csv
regions.sort_values( 'parent_region_code', inplace=True )
regions.to_csv( 'un_regions.csv', index=False )
regions.sample(10)

,region_code,region_name,parent_region_code
26,030,Eastern Asia,142
6,154,Northern Europe,150
8,015,Northern Africa,002
20,151,Eastern Europe,150
22,021,Northern America,019
21,013,Central America,419
11,017,Middle Africa,202
7,039,Southern Europe,150
14,029,Caribbean,419
17,145,Western Asia,142


In [71]:
un_groupings.to_csv( 'groupings.csv', index=False )
un_groupings

,society_group_name,society_group_type
0,Least Developed Countries,1
1,Landlocked Developing Countries,1
2,Small Island Developing States,1
3,Developed Countries,1
4,Developing Countries,1


In [72]:
country_to_group.to_csv( 'country_to_group.csv', index=False )
country_to_group.sample(10)

,country_id,group_id
221,33,4
63,130,1
34,197,0
158,98,3
106,143,2
271,103,4
27,149,0
20,120,0
97,91,2
369,236,4


<a id='db'></a>
# Insert Into Database

In [73]:
# Import classes and functions needed for this analysis from config module
# These are only available on my computer
from config import dbaccess, build_insert_query, error_dict_to_string, un_region, society, society_group

# Create an instance of the DBAccess class for running queries
db = dbaccess.DBAccess()

## UN Regions
### Code

In [74]:
# Check for errors in the UN Regions dataframe before insertion
print( error_dict_to_string( un_region.UNRegion.validate_regions(regions, db) ) )

No errors


In [75]:
# Truncate table each run to prevent collisions
# REMOVE WHEN DONE WITH NOTEBOOK
db.truncate_table( 'un_m49_region' )

# Insert UN Regions
print( 'Expecting to insert:', regions.shape[0], 'rows' ) # How many rows am I expecting to be inserted?
insert_un_regions = db.run_query( build_insert_query('un_m49_region', regions) )
print( insert_un_regions['rows'], 'records inserted' )

Expecting to insert: 31 rows
31 records inserted


### Test

In [76]:
un_regions_query = db.run_query( 'SELECT * FROM un_m49_region' )
un_regions = pd.DataFrame.from_dict( un_regions_query['data'] ).set_index('region_code')
un_regions.sample(5)

,region_name,parent_region_code
region_code,,
830,Channel Islands,154
151,Eastern Europe,150
035,South-eastern Asia,142
017,Middle Africa,202
013,Central America,419


## UN Groupings
### Code

In [77]:
# Check for errors in the Society Groups dataframe before insertion
print( error_dict_to_string( society_group.SocietyGroup.validate_society_groups(un_groupings, db) ) )

No errors


In [78]:
# Truncate table each run to prevent collisions and duplicates
# REMOVE WHEN DONE WITH NOTEBOOK
db.truncate_table( 'society_group' )

print( 'Expecting to insert:', un_groupings.shape[0], 'rows' ) # How many rows am I expecting to be inserted?
insert_groupings = db.run_query( build_insert_query('society_group', un_groupings) )
print( insert_groupings['rows'], 'records inserted' )

Expecting to insert: 5 rows
5 records inserted


### Test

In [79]:
# Get all records in the society_group table and set into a dataframe
society_groups_query = db.run_query( 'SELECT * FROM society_group' )
society_groups = pd.DataFrame.from_dict( society_groups_query['data'] ).set_index('society_group_id')
society_groups.sample(5)

,society_group_name,society_group_type
society_group_id,,
4,Developed Countries,1
2,Landlocked Developing Countries,1
5,Developing Countries,1
3,Small Island Developing States,1
1,Least Developed Countries,1


## Countries (Societies)
Insert countries into `society` table. 

### Code

In [84]:
# Check for errors in the Countries dataframe before insertion
print( error_dict_to_string( society.Society.validate_societies( countries, db ) ) )

No errors


In [85]:
# Truncate table each run to prevent collisions and duplicates
# REMOVE WHEN DONE WITH NOTEBOOK
db.truncate_table( 'society' )

# Insert Societies
print( 'Expecting to insert:', countries.shape[0], 'rows' ) # How many rows am I expecting to be inserted?
insert_countries = db.run_query( build_insert_query('society', countries) )
print( insert_countries['rows'], 'records inserted' )

Expecting to insert: 249 rows
249 records inserted


### Test

In [86]:
# Get all records in the society table and set into a dataframe
societies_query = db.run_query( 'SELECT * FROM society' )
societies = pd.DataFrame.from_dict( societies_query['data'] ).set_index('society_id')
societies.sample(5)

,common_name,official_name,capital,society_type_id,un_m49,iso_alpha3,un_region
society_id,,,,,,,
137,Martinique,Martinique,Fort-de-France,1,474,MTQ,029
40,Canada,Canada,Ottawa,1,124,CAN,021
202,Somalia,Federal Republic of Somalia,Mogadishu,1,706,SOM,014
229,Turks and Caicos Islands,Turks and Caicos Islands,Cockburn Town,1,796,TCA,029
22,Belize,Belize,Belmopan,1,084,BLZ,013


## Countries to Groups

### Code

### Test